In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
from pytc import PyTC
from pes import PES
from eigenvector_follow import eigenvector_follow
from psiw import *
from casci_lot_svd import CASCI_LOT_SVD
from nifty import pvec1d,pmat2d,click,printcool
from molecule import Molecule
from se_cross import SE_Cross
import manage_xyz
from penalty_pes import Penalty_PES

In [2]:
##### => Job Data <= #####
states = [(1,0),(1,1)]
charge=0
nocc=7
nactive=2
basis='6-31gs'
filepath = '../data/ethylene.xyz'

In [3]:

#### => PSIW Obj <= ######
printcool("Build resources")
resources = ls.ResourceList.build()
printcool('{}'.format(resources))

molecule = ls.Molecule.from_xyz_file(filepath)
geom = geometry.Geometry.build(
    resources=resources,
    molecule=molecule,
    basisname=basis,
    )
printcool('{}'.format(geom))

ref = RHF.from_options(
     geometry= geom, 
     g_convergence=1.0E-6,
     thre_sp=1.0E-15,
     grad_thre_dp=1.0E-7,
     diis_max_vecs=10,
     fomo=True,
     fomo_method='gaussian',
     fomo_temp=0.3,
     fomo_nocc=nocc,
     fomo_nact=nactive,
     print_level=1,
    )
ref.compute_energy()
casci = CASCI.from_options(
    reference=ref,
    nocc=nocc,
    nact=nactive,
    grad_thre_dp=1.0E-7,
    grad_thre_sp=1.0E-16,
    thre_dp = 1.0E-7,  
    thre_sp=1.0E-16,
    nalpha=nactive/2,
    nbeta=nactive/2,
    S_inds=[0],
    S_nstates=[2],
    print_level=1,
    )
casci.compute_energy()

psiw = CASCI_LOT_SVD.from_options(
    casci=casci,
    svd_reference=casci,  
    rhf_guess=True,
    rhf_mom=True,
    orbital_coincidence='core',
    state_coincidence='full',
    )

#========================================================#
#|                   Build resources                    |#
#========================================================#
#==============================================================#
#|                       ResourceList:                        |#
#|                       CPU threads:   4                     |#
#|                       GPU contexts:  1                     |#
#|                     GPU context details:                   |#
#|    N  ID                 Name  CC  CPU Buffer  GPU Buffer  |#
#|    0   0    GeForce GTX TITAN 3.5        1024        1024  |#
#|                                                            |#
#==============================================================#
#========================================================#
#|                      Geometry:                       |#
#|                      QMMM = False                    |#
#|                      -D3  = False                    |#
#|

In [4]:
####### =>  Build the pyGSM objects <= #########
# level of theory
printcool("Build the pyGSM Level of Theory object (LOT)")
lot=PyTC.from_options(states=[(1,0),(1,1)],extra_kwargs={'psiw':psiw},do_coupling=False,fnm=filepath) 

# => Create PES objects <= #
printcool("Building the PES objects")
pes1 = PES.from_options(lot=lot,ad_idx=states[0][1],multiplicity=states[0][0])
pes2 = PES.from_options(lot=lot,ad_idx=states[1][1],multiplicity=states[1][0])
pes = Penalty_PES(pes1,pes2,lot)

# => Molecule <= #
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
M = Molecule.from_options(fnm=filepath,PES=pes,coordinate_type="TRIC")

# => Optimizer object <= #
ef = eigenvector_follow.from_options(DMAX=0.05,print_level=1) #Linesearch=NoLineSearch

# => Create GSM object <= #
printcool(" Building the GSM object")
driving_coords = [('TORSION',2,1,4,6,120.),('TORSION',2,1,4,5,180.)] 
#extra torsion here to ensure proper orientation w.r.t et_meci
gsm = SE_Cross.from_options(reactant=M,nnodes=20,driving_coords=driving_coords,DQMAG_MAX=0.5,BDIST_RATIO=0.75,optimizer=ef)

#========================================================#
#|     Build the pyGSM Level of Theory object (LOT)     |#
#========================================================#
 initializing LOT from file
#========================================================#
#|               Building the PES objects               |#
#========================================================#
 PES1 multiplicity: 1 PES2 multiplicity: 1
#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 making primitives from options!
 making primitive Hessian
 forming Hessian in basis
#========================================================#
#|                Building the GSM object               |#
#========================================================#

   

In [5]:
# => Run GSM <= #
print ' Starting GSM '
gsm.go_gsm(opt_steps=5)

 Starting GSM 
#========================================================#
#|                 Doing SE-MECI search                 |#
#========================================================#
 Initial energy is -48629.4844
 current torv: 0.000 align to 120.000 diff(deg): 120.000
 current torv: -3.142 align to 180.000 diff(deg): -0.000
 Initial bdist is 2.094
 Adding reactant node
 adding node: 1 from node 0
 current torv: 0.000 align to 120.000 diff(deg): 120.000
 current torv: -3.142 align to 180.000 diff(deg): -0.000
 dqmag: 0.500 from bdist: 2.094
 dq0[constraint]: -0.500
 Iter: 1 Err-dQ = 3.01890e-02 RMSD: 3.89438e-02 Damp: 1.00000e+00
 Iter: 2 Err-dQ (Best) = 4.57754e-04 (3.01890e-02) RMSD: 7.37627e-03 Damp: 1.00000e+00 (Good)
 Iter: 3 Err-dQ (Best) = 5.48213e-08 (4.57754e-04) RMSD: 1.00845e-04 Damp: 1.00000e+00 (Good)
 Cartesian coordinates obtained after 3 microiterations (rmsd = 1.008e-04 |dQ| = 5.482e-08)
 initializing LOT from file
 PES1 multiplicity: 1 PES2 multiplicity: 1
 

In [6]:
# => post processing <= #
manage_xyz.write_xyz('et_meci.xyz',gsm.nodes[gsm.nR].geometry,scale=1.)

#Save a npz file
Cocc= ls.Tensor.array(gsm.nodes[gsm.nR].PES.lot.psiw.casci.reference.tensors['Cocc'])
Cact= ls.Tensor.array(gsm.nodes[gsm.nR].PES.lot.psiw.casci.reference.tensors['Cact'])
D   = ls.Tensor.array(gsm.nodes[gsm.nR].PES.lot.psiw.casci.reference.tensors['D'])
np.savez(
    'Cmom.npz',
    Cocc=Cocc,
    Cact=Cact,
    D=D
    )